## 예제 뉴스그룹 LDA(LSA 동일 예제)

In [23]:
!pip install stopwords

You should consider upgrading via the 'C:\Users\Lee\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

dataset.target_names # target_name에는 카테고리가 표시

news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

C:\Users\alsgu\AppData\Local\Temp/ipykernel_19772/2333237559.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [2]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alsgu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [4]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

In [5]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.061*"space" + 0.022*"nasa" + 0.014*"launch" + 0.011*"satellite"')
(1, '0.014*"would" + 0.010*"people" + 0.009*"like" + 0.008*"know"')
(2, '0.015*"government" + 0.015*"state" + 0.014*"states" + 0.013*"rights"')
(3, '0.016*"windows" + 0.015*"system" + 0.015*"problem" + 0.013*"drive"')
(4, '0.019*"data" + 0.006*"analysis" + 0.006*"design" + 0.006*"system"')
(5, '0.018*"radar" + 0.014*"icon" + 0.013*"detector" + 0.011*"colour"')
(6, '0.011*"people" + 0.010*"would" + 0.009*"think" + 0.007*"believe"')
(7, '0.026*"game" + 0.024*"team" + 0.017*"year" + 0.017*"games"')
(8, '0.012*"israel" + 0.009*"israeli" + 0.008*"much" + 0.006*"good"')
(9, '0.012*"jesus" + 0.008*"jews" + 0.007*"world" + 0.006*"church"')
(10, '0.025*"health" + 0.020*"medical" + 0.014*"disease" + 0.011*"pain"')
(11, '0.026*"encryption" + 0.021*"chip" + 0.020*"keys" + 0.019*"government"')
(12, '0.018*"period" + 0.013*"power" + 0.011*"drive" + 0.011*"play"')
(13, '0.010*"cover" + 0.007*"picture" + 0.006*"plastic" + 0.006*"

#### LDA 시각화

In [9]:
# LDA를 시각화 할 수 있는 패키지
!pip install pyLDAvis

In [12]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)
# 원이 가장 크다는 것은 토픽에 해당되는 텍스트가 많다는 뜻임
# 거리에 따라 서로 관련성을 나타냄
# 푸른색까지는 전체 기사에서 나오는 비중, 빨간색은 해당되는 토픽에서 등장하는 비중
# 빨간색이 많고 파란색이 적을 수록 해당 토픽을 알기 쉬움

#### 문서별 토픽 확인하기

In [13]:
# lsa와 lda의 차이점 - 
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [14]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,8.0,0.2652,"[(1, 0.18108119), (2, 0.08787103), (6, 0.20538..."
1,1,6.0,0.6581,"[(6, 0.6581477), (7, 0.047038857), (8, 0.07553..."
2,2,1.0,0.7094,"[(1, 0.7094395), (4, 0.04704907), (8, 0.156211..."
3,3,1.0,0.3144,"[(1, 0.31439263), (6, 0.15986922), (9, 0.06199..."
4,4,6.0,0.4742,"[(1, 0.084518805), (6, 0.4742311), (7, 0.18443..."
5,5,9.0,0.5242,"[(1, 0.31991148), (7, 0.06749553), (9, 0.52424..."
6,6,8.0,0.3232,"[(3, 0.15880163), (4, 0.026154496), (6, 0.1227..."
7,7,1.0,0.3827,"[(1, 0.3827157), (8, 0.23005284), (9, 0.3732826)]"
8,8,9.0,0.3797,"[(1, 0.3050859), (6, 0.061881293), (9, 0.37969..."
9,9,1.0,0.3877,"[(1, 0.38774347), (9, 0.028413076), (14, 0.285..."
